## import pymemcache library

In [1]:
import time
from pymemcache.client.base import Client
from datetime import datetime

## initialize client

In [2]:
client = Client('localhost')

## test client.version command

This command sends b"version\r\n" command.

Original memcache returned following:
b"VERSION 1.6.36"
client.version() parsed it as:
b'1.6.36'
And Goldfish will return b'VERSION Goldfish 1.0'

In [3]:
version = client.version()
assert version.decode('utf-8').startswith('Goldfish'), "does not return the version in right format"
version

b'Goldfish 1.0'

## client.set and client get commands



In [4]:
# First lets test the simple usecase

In [5]:
key = 'key1'
value = 'value1'
client.set(key, value)
server_value = client.get(key)
assert server_value.decode('utf-8') == value , "server returned a different error than expected"
server_value

b'value1'

In [6]:
# Now we try the same call with noreply=False and flag values

In [7]:
key = 'key2'
value = 'value2'
client.set(key, value, noreply=False, flags=1)
server_value = client.get(key)
assert server_value.decode('utf-8') == value , "server returned a different error than expected"
server_value

b'value2'

In [8]:
# Now lets test what happens when we set expiration

In [11]:
key = 'key3'
value = 'value3'
client.set(key, value, noreply=False, flags=1, expire=1)
server_value = client.get(key)
assert server_value.decode('utf-8') == value , "server returned a different error than expected"
time.sleep(1)
server_value2 = client.get(key)
assert server_value2 is None, "Value did not expired"
print(f'server value before expiration = {server_value}')
print(f'server value after expiration  = {server_value2}')

server value before expiration = b'value3'
server value after expiration  = None


In [9]:
# Lastly I want to try sending 100000 key, value pairs to get the server underload

In [12]:
value_count = 100000
def load_test(enable_print=False):
    enable_print and print('starting to enter values', datetime.now())
    for i in range(value_count):
        key = f'key{i}'
        value = f'value{i}'
        client.set(key, value)
    enable_print and print('value_entry is complete. starting to check the values returning from server', datetime.now())
    for i in range(value_count):
        enable_print and print(i)
        key = f'key{i}'
        value = f'value{i}'
        enable_print and print('precall')
        server_value = client.get(key)
        enable_print and print('postcall', server_value)
        assert value == server_value.decode('utf8')
    enable_print and print('test is complete', datetime.now())

In [16]:
# This is goldfish load test
client = Client('localhost')
value_count = 100000
%time load_test()

CPU times: user 5.31 s, sys: 3.46 s, total: 8.77 s
Wall time: 43.2 s


In [15]:
# This is original memcached load test
client = Client('localhost')
value_count = 100000
%time load_test()

CPU times: user 3.25 s, sys: 2.05 s, total: 5.29 s
Wall time: 10.7 s


In [17]:
# It seems like goldfish is curently 4 times slower than original memcached.

## test client.quit command

This command breakes the tcp connection with the server.
client.quit() does not give any feedback

In [18]:
client.quit()